Выполнила: Анастасия Плавина

## Тема 3. Коллаборативная фильтрация

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items

In [2]:
# положила в отдельный файл, тут оставила для проверки

def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    # берем самую последнюю покупку каждого айтема
    old_items = data.groupby(by=['item_id'])['item_id', 'week_no'].max()
    # получаем айтемы, которые были куплены последний раз больше года назад
    old_items = list(old_items[old_items['week_no'] <= (data['week_no'].max() - 52)].index)
    data = data[~data['item_id'].isin(old_items)]

    # Уберем не интересные для рекоммендаций категории (department), уберем товары купленные менее 1000 раз
    top_departments = data['department'].value_counts()[lambda x: x > 1000].index
    data = data[data['department'].isin(top_departments)]
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    # полагаем, что цены на товары в долларах, переведем в рубли по курсу 74.1, чтобы потом отфильтровать все товары ниже стоимости привлечения
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data['price_rub'] = data['price'] * 74.1
    data = data[data['price_rub'] > 60]
    
    # Уберем слишком дорогие товары
    data = data[data['price'] < data['price'].quantile(0.99995)]
    
    return data 
  

### Загрузка датасетов, префильтрация, деление на трейн и тест

In [3]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
data = data.merge(item_features, on='item_id')

# префильтрация
data = prefilter_items(data)

test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_train.head(3)

<ipython-input-2-5dd03416ca70>:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  old_items = data.groupby(by=['item_id'])['item_id', 'week_no'].max()


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price,price_rub
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,1.39,102.999
1,1929,27021022215,4,1004906,1,1.39,441,-0.6,1755,1,0.0,0.0,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,1.39,102.999
2,568,27021090189,4,1004906,1,1.39,446,-0.6,35,1,0.0,0.0,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,1.39,102.999


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[1004906, 1082185, 1081189, 1041796, 940947, 9..."
1,3,"[1053690, 994891, 1096727, 1083328, 872021, 83..."


### Подготовка матрицы для модели 

In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_10000 = popularity.sort_values('n_sold', ascending=False).head(10000).item_id.tolist()

In [6]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_10000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,34873,117847,201704,279994,818980,818981,818996,819063,819112,819255,...,15926927,15972074,15972115,15972298,15972565,15972790,16100266,16729299,16729415,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

### ALS с bm25 взвешиванием
т.к. пока это лучший скор

In [8]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [9]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [10]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
print(f"Precision_@k': {result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean():.4f}")


Precision_@k': 0.1873
CPU times: user 1min 39s, sys: 26.5 s, total: 2min 5s
Wall time: 59 s


### Подбор параметров

In [11]:
param_grid = {'num_factors': [40, 80, 100, 120],
              'regularization': [0.0, 1e-5, 1e-1, 1e1],
              'iterations': [5, 10, 15]}

grid_result = {'factors': [],
               'regularization': [], 
               'iterations': [],
               'precision@k': []}

In [12]:
%%time


for param in itertools.product(*param_grid.values()):
    print('Parameters:', param)
    model = AlternatingLeastSquares(factors=param[0], 
                                regularization=param[1],
                                iterations=param[2], 
                                calculate_training_loss=True, 
                                num_threads=4, random_state=42)
    
    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)
    
    
    result['als_grid'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

    grid_result['factors'].append(param[0])
    grid_result['regularization'].append(param[1])
    grid_result['iterations'].append(param[2])
    grid_result['precision@k'].append(result.apply(lambda row: precision_at_k(row['als_grid'], row['actual']),
                                                   axis=1).mean())
     

Parameters: (40, 0.0, 5)



Parameters: (40, 0.0, 10)



Parameters: (40, 0.0, 15)



Parameters: (40, 1e-05, 5)



Parameters: (40, 1e-05, 10)


...


Parameters: (120, 0.1, 15)



Parameters: (120, 10.0, 5)



Parameters: (120, 10.0, 10)



Parameters: (120, 10.0, 15)



CPU times: user 1h 3min 57s, sys: 14min 59s, total: 1h 18min 56s
Wall time: 38min 16s


In [13]:
df = pd.DataFrame(grid_result).sort_values('precision@k', axis=0, ascending=False).reset_index(drop=True)
best_params = df.iloc[0].to_dict()
best_params

{'factors': 100.0,
 'regularization': 10.0,
 'iterations': 5.0,
 'precision@k': 0.20138339920948617}

### Финальный результат

In [14]:
%%time

model = AlternatingLeastSquares(factors=int(best_params['factors']), 
                                regularization=best_params['regularization'],
                                iterations=int(best_params['iterations']), 
                                calculate_training_loss=True, 
                                num_threads=4, random_state=42)
    
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


result['als_best_params'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als_best_params'], row['actual']), axis=1).mean()


CPU times: user 1min 14s, sys: 13.6 s, total: 1min 27s
Wall time: 40.6 s


0.20138339920948617

Подбор основных параметров дал небольшое улучшение метрики.

In [15]:
result.to_csv('predictions_mf.csv', index=False)

-----

# Production - ответы на вопросы

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его сущесвтенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель) 

__Оговариваем с продажами. Значимость прироста можем проверить АБ-тестированием.__


- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)

__Предположу варианты эффектов - рост средних значений Revenue per user, Conversion per session, Conversion per user. Возможно, изначально мы целимся на определенный сегмент и хотим повышение метрик именно в нем. Например, мы планируем увеличить конверсию среди возрастных клиентов или недавно зарегистрированных пользователей. Также мы можем хотеть повысить продажи какой-то определенной группы товаров - здесь нужно настраивать выборку юзеров, по характеристикам, которые у нас есть, чтобы понять, кому лучше зайдет определенный товар.__

- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они? 

__Важно определиться с актуальностью имейла до начала оценки модели, т.к. нужно точно понимать, рассылка не имеет эффекта из-за содержания или неакутальности адреса. Предположу, что можно настроить проверку открытия письма по обращению по ссылке изображения в теле рассылки. Если письмо открывалось, считаем имейл актуальным (это фантазии, но слышала так можно).__

- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?

__Есть вероятность, что юзер совершит оттток от назойливого воздействия. Поэтому для начала неплохо сделать аплифт моделирование и более активное воздействие пробовать на клиентах с высокой вероятностью конверсии после воздействия. Вариант печати на чеке выглядит безопасно, однако надо просчитать стоимость печати на каждом чеке.__


- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)

__Сложилось впечатление, что ранжированные рекомендации более эффективны, т.к. даже очень удачные рекомендации, но в неправильном порядке, могут не сработать. N топа определять лучше тоже эксперементально по метрикам на тестовой выборке.__

- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?

__Акционные товары могут заходить всем, но нужно фильтровать категории и ценовые сегменты в соответствии с сегментом пользователей. Например, возрастным пользователям меньше предлагать детские товары (памперсы и т.п.). Хотя это уже задача модели, но какие-то базовые вещи можно сразу настроить, если база данных по пользователю позволяет.__

- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки

__Готовы потратить такое число денег, которое даст повышение y% прибыли по сравнению с работой ресвиса без рекомендательной системы.__

- Cколько мы хотим зарабатывать с одного привлеченного юзера?
__Больше CAC (т.е. того числа, которое было потрачено на его привлечение)__

---
- А точно нужно сортировать по вероятности?
__Тоже можно проверить a/b-тестом.

- Какую метрику использовать?

__Если не ранжируем и целимся на повышение метрик юзера, а не продаж товара, то precision@k представляется универсальной метрикой.__

- Сколько раз в неделю отпрпавляем рассылку?

__Выясняем по метрике экспериментально, если позволяют ресурсы__

- В какое время отправляем рассылку?

__Можно отправлять за несколько часов до среднего по юзеру времени посещения магазина. Как вариант, брать время, в которое на сайте максимальное количество юзеров и считать его универсальным для всех.__

- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?

__Можно по разному ранжировать + добавлять что-то из купленных товаров или популярных товаров__

- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?

__Если не целимся на повышение продаж конкретной категории, то разные товары нужны. Как минимум, можно брать из разных ценовых сегментов или разных категорий, чтобы получить подборку разных товаров.__
